In [1]:
import getpass
import email

In [2]:
import ssl

from socks import create_connection
from socks import PROXY_TYPE_SOCKS4
from socks import PROXY_TYPE_SOCKS5
from socks import PROXY_TYPE_HTTP

from imaplib import IMAP4
from imaplib import IMAP4_PORT
from imaplib import IMAP4_SSL_PORT

class SocksIMAP4SSL(SocksIMAP4):

    def __init__(self, host='', port=IMAP4_SSL_PORT, keyfile=None, certfile=None, ssl_context=None, proxy_addr=None,
                 proxy_port=None, rdns=True, username=None, password=None, proxy_type="socks5"):

        if ssl_context is not None and keyfile is not None:
                raise ValueError("ssl_context and keyfile arguments are mutually "
                                 "exclusive")
        if ssl_context is not None and certfile is not None:
            raise ValueError("ssl_context and certfile arguments are mutually "
                             "exclusive")

        self.keyfile = keyfile
        self.certfile = certfile
        if ssl_context is None:
            ssl_context = ssl._create_stdlib_context(certfile=certfile,
                                                     keyfile=keyfile)
        self.ssl_context = ssl_context

        SocksIMAP4.__init__(self, host, port, proxy_addr=proxy_addr, proxy_port=proxy_port,
                            rdns=rdns, username=username, password=password, proxy_type=proxy_type)

    def _create_socket(self):
        sock = SocksIMAP4._create_socket(self)
        server_hostname = self.host if ssl.HAS_SNI else None
        return self.ssl_context.wrap_socket(sock, server_hostname=server_hostname)

    def open(self, host='', port=IMAP4_PORT):
        SocksIMAP4.open(self, host, port)

In [9]:
import configparser

In [133]:
conf = configparser.ConfigParser()
conf.read('im.conf')

account = conf.get('email', 'account')
password = conf.get('email', 'password')
imap_server = conf.get('email', 'server')
imap_port = conf.getint('email', 'port')

# setup proxy in case of some email servers cannot be accessed.
proxy_addr = conf.get('proxy', 'host')
proxy_port = conf.getint('proxy', 'port')
proxy_type = conf.get('proxy', 'type')

mailbox = SocksIMAP4SSL(host=imap_server, port=imap_port,
                        proxy_addr=proxy_addr, proxy_port=proxy_port, proxy_type=proxy_type)
mailbox.login(account, password)

('OK', [b'LOGIN completed'])

In [134]:
mailbox.select("INBOX")

('OK', [b'2'])

In [135]:
def decoder(msg):
    '''
    Decode msg like this format:
    (b'\xbf\xa1\xe8\xa1\x8c\xe4\xb8\xba', 'utf-8')
    '''
    payload = msg[0]
    charset = msg[1]
    
    if charset is None:
        # no need to decode:
        return payload
    else:
        return payload.decode(charset)

In [136]:
def parse_email(msg):
    mail_content = None
    content_type = None
    
    for part in msg.walk():  # walk(): used to dfs iterate the message tree object
        if not part.is_multipart(): 
            # .is_multipart()
            # Return True if the message’s payload is a list of sub-Message objects,  
            # otherwise return False. When is_multipart() returns False, 
            # the payload should be a string object.  
            content_type = part.get_content_type()  
            charset = part.get_content_charset()  
            if charset == None or charset == 'utf-8':  
                return part.get_payload()
            else:  
                return unicode(part.get_payload(decode=True),charset).encode('utf-8')   

In [137]:
# fetch all mails in the inbox
typ, data = mailbox.search(None, 'ALL')
for num in data[0].split():
    typ, data = mailbox.fetch(num, '(RFC822)')
    msg = email.message_from_string(data[0][1].decode('utf-8'))
    subject = decoder(email.header.decode_header(msg['Subject'])[0])
    from_sender = msg['From'].split()[-1].replace('<', '').replace('>', '')
    to_sender = msg['To'].split()[-1].replace('<', '').replace('>', '')
    date = msg['Date']
    content = parse_email(msg)
    print("Subject: ", subject)
    print("From: ", from_sender)
    print("To: ", to_sender)
    print("Date: ", date)
    print('*'*100)
    print(content)
    print('*'*100)
    print('-'*100)

Subject:  网易邮箱提醒：一次被阻止的收信行为
From:  mail@service.netease.com
To:  pengym_111@163.com
Date:  Mon, 11 Dec 2017 23:38:59 +0800 (CST)
****************************************************************************************************
<meta charset=3D"UTF-8" />
<table align=3D"center" style=3D"font-family:Microsoft YaHei,Simsun;width:5=
40px;table-layout:fixed;" bgcolor=3D"#ffffff" cellpadding=3D"0" cellspacing=
=3D"0">
<tbody>
    <tr>
        <td style=3D"display:none;"></td>
</tr>
<tr>
    <td>
        <table style=3D"width:540px;" border=3D"0" cellpadding=3D"0" cellsp=
acing=3D"0">
<tr>
    <td style=3D"width:540px;height:100px;font-family:Microsoft YaHei,Simsu=
n;background:#ffffff;"><img src=3D"http://mimg.127.net/hxm/dashi/external/n=
otify/14851528754240068.png" style=3D"display:block;border:0;width: 540px;h=
eight: 100px;"/></td>
</tr>
</table>
</td>
</tr>
<tr>
    <td>
        <table style=3D"width:540px;" border=3D"0" cellpadding=3D"0" cellsp=
acing=3D"0">
<tr>
    <td style=3D"w

In [ ]:
mailbox.close()
mailbox.logout()